# Парсинг данных eLibrary для проекта "Паттерны коллаборации в Российском социологическом сообществе"

В этом файле производится выгрузка данных eLibrary для проекта Лаборатории. Осуществляется импорт необходимых библиотек, загружаются id необходимых к выгрузке статьи, производится сама выгрузка основного массива данных.

Основной код выгрузки был подготовлен Львом Ципесом, в выгрузке данных участвовали Василиса Ващенко и Лика Капустина. Выгрузка производилась в июне-июле 2022 года.

Эта версия была закончена и прокомментирована Ликой Капустиной.

## Подготовка к работе

### Загрузка необходимых библиотек

In [2]:
#pip install xmltodict
from numpy import fromstring # приведение данных к строке
import requests # модуль для запросов к данным по ссылкам
import xmltodict # работа с xml
import pandas as pd # модуль для работы с таблицами
from tqdm import tqdm # модуль для работы с tqdm
from time import sleep # импортируем функцию сна 
import warnings # модуль с предупреждениями
warnings.filterwarnings("ignore") # выбираем игнорировать предупреждения
import xml.etree.ElementTree as ET # работа с xml
from xml.etree.ElementTree import parse # работа с xml
from urllib.request import urlopen # работа с запросами
import pickle # формат данных pickle

### Проверка ip-адреса

Выгрузка данных могла производиться только по рабочему ip-адресу Дарьи Васильевны в корпусе Вышки на Покровском бульваре, поэтому перед выгрузкой данных осуществлялось подключение к ip-адресу Дарьи Васильевны.

In [3]:
import http.client
conn = http.client.HTTPConnection("ifconfig.me")
conn.request("GET", "/ip")
conn.getresponse().read()

b'92.242.58.41'

Загружаем текстовый файл с id публикаций.

In [5]:
with open('421_1.txt') as f: #заменить на свой адрес файла
    lines = f.readlines()

Проверяем длину файла. Если id'шников 66 473 – все хорошо. Действительно, по `True` видим, что все хорошо.

In [6]:
a = len(lines)
print(a == 66473)

True


## Получение списка отсутствующих id статей

Эта версия файла с парсингом данных – 2.5, и до этого нами (Лев, Василиса, Лика) производлилось несколько попыток вгурзки данных: из-за технических проблем и иных сложностей код несколько раз дорабатывался. Сейчас мы загружаем файл с попыткой выгрузки от 6 июля, избавляемся от дупликатов (статей, которые по каким-то причинаам техничексого характера дважды могли выгрузиться из eLibrary) и смотрим на получившиеся данные.

In [10]:
july_6_data = pd.read_excel('6_july_try.xlsx', index_col=0)
july_6_data

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,12996402,4,700413,1107,Демченко,Т.С.,СГА,none,none,none,...,4,3,0,33385645,NaN,1,1,http://elibrary.ru/item.asp?id=12996402,NaN,"Демченко, Т.С. ЭМПИРИЧЕСКОЕ ИССЛЕДОВАНИЕ ФОРМ ..."
1,13033127,4,548357,NaN,Вехов,Иван Васильевич,TNS Россия,none,none,none,...,4,0,0,33387314,NaN,1,1,http://elibrary.ru/item.asp?id=13033127,NaN,"Вехов, И.В. САМЫЕ ПОПУЛЯРНЫЕ ТЕЛЕПРОГРАММЫ 200..."
2,13033538,4,321778,3453,Жигунова,Галина Владимировна,Мурманский государственный технический универс...,none,none,none,...,4,17,2,33387324,NaN,1,1,http://elibrary.ru/item.asp?id=13033538,NaN,"Жигунова, Г.В. Толерантность как ценность граж..."
3,13033945,4,0,none,Исаков,Анатолий Исаевич,none,Isakov,Anatoly Isaakovich,none,...,4,1,0,33387342,NaN,1,0,http://elibrary.ru/item.asp?id=13033945,none,"ИСАКОВ, А.И. Вместе с армией и народом / А.И. ..."
4,13034149,4,673159,4812,Лысенко,Екатерина Григорьевна,Российский государственный социальный университет,none,none,none,...,4,12,0,33387349,NaN,1,0,http://elibrary.ru/item.asp?id=13034149,none,"Лысенко, Е.Г. ПРОБЛЕМА МАРГИНАЛЬНОСТИ В ЗАПАДН..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,18842441,4,270726,none,Бачинин,Владислав Аркадьевич,none,Bachinin,V.A.,none,...,4,7,0,33820602,NaN,1,0,NaN,none,"Бачинин, В.А. СОЦИОЛОГИЯ ЛИТЕРАТУРЫ КАК ПРОСТР..."
20996,18811564,4,137142,1790,Бюраева,Юлия Григорьевна,Бурятский научный центр СО РАН,Byuraeva,Yu. G.,none,...,4,4,0,33819528,NaN,1,0,NaN,none,"Бюраева, Ю.Г. Социально-трудовой потенциал сел..."
20997,18786053,4,0,NaN,Басаргина,Е.В.,Государственное образовательное учреждение сре...,none,none,none,...,4,0,0,33818626,NaN,1,1,NaN,NaN,"Басаргина, Е.В. МЕЖНАЦИОНАЛЬНЫЕ ОТНОШЕНИЯ В СИ..."
20998,18337362,4,0,NaN,Филиппов,В.В.,ООО «Реставрационные музейные мастерские»,none,none,none,...,4,0,0,33799317,NaN,1,1,NaN,NaN,"Филиппов, В.В. ХАКАССКИЙ ОБЛАСТНОЙ (ЧЕРНОГОРСК..."


In [12]:
first_part = july_6_data.drop_duplicates('id')
first_part

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,12996402,4,700413,1107,Демченко,Т.С.,СГА,none,none,none,...,4,3,0,33385645,NaN,1,1,http://elibrary.ru/item.asp?id=12996402,NaN,"Демченко, Т.С. ЭМПИРИЧЕСКОЕ ИССЛЕДОВАНИЕ ФОРМ ..."
1,13033127,4,548357,NaN,Вехов,Иван Васильевич,TNS Россия,none,none,none,...,4,0,0,33387314,NaN,1,1,http://elibrary.ru/item.asp?id=13033127,NaN,"Вехов, И.В. САМЫЕ ПОПУЛЯРНЫЕ ТЕЛЕПРОГРАММЫ 200..."
2,13033538,4,321778,3453,Жигунова,Галина Владимировна,Мурманский государственный технический универс...,none,none,none,...,4,17,2,33387324,NaN,1,1,http://elibrary.ru/item.asp?id=13033538,NaN,"Жигунова, Г.В. Толерантность как ценность граж..."
3,13033945,4,0,none,Исаков,Анатолий Исаевич,none,Isakov,Anatoly Isaakovich,none,...,4,1,0,33387342,NaN,1,0,http://elibrary.ru/item.asp?id=13033945,none,"ИСАКОВ, А.И. Вместе с армией и народом / А.И. ..."
4,13034149,4,673159,4812,Лысенко,Екатерина Григорьевна,Российский государственный социальный университет,none,none,none,...,4,12,0,33387349,NaN,1,0,http://elibrary.ru/item.asp?id=13034149,none,"Лысенко, Е.Г. ПРОБЛЕМА МАРГИНАЛЬНОСТИ В ЗАПАДН..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38569,23018570,4,811563,305,Давлетшина,Гульнара Рафисовна,ФГБОУ ВПО «Башкирский государственный медицинс...,none,none,none,...,4,3,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018570,NaN,"Давлетшина, Г.Р. ТРУДНОСТИ АДАПТАЦИИ ИНДИВИДОВ..."
38570,23018587,4,415461,804,Прокшина,Лира Рифгатовна,Башкирский институт социальных технологий (фил...,none,none,none,...,4,0,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018587,NaN,"Прокшина, Л.Р. ИГРА КАК АДАПТАЦИОННЫЙ ПОТЕНЦИА..."
38571,23025275,4,0,211,Гареева,Лена Фларитовна,Башкирский государственный университет,Gareeva,L.F.,none,...,4,0,0,34054658,NaN,1,0,http://elibrary.ru/item.asp?id=23025275,none,"Гареева, Л.Ф. Социально-философские проблемы с..."
38572,23027380,4,215177,7082,Васильева,Е.И.,Российская академия народного хозяйства и госу...,Vasilyeva,E.I.,Russian Presidential Academy of National Econo...,...,4,0,0,34054790,NaN,0,0,http://elibrary.ru/item.asp?id=23027380,none,"Васильева, Е.И. Качество и доступность предост..."


В ячейках ниже определяются id статей, которые необходимо довыгрузить.

In [20]:
id_in_first_part = [str(i) for i in first_part['id']]
id_in_first_part[0:10]

['12996402',
 '13033127',
 '13033538',
 '13033945',
 '13034149',
 '13034174',
 '13050100',
 '13060428',
 '13071231',
 '13088897']

In [21]:
need_id = [i[:8] for i in lines if i[:8] not in id_in_first_part]
need_id[0:10]

['23064934',
 '23049589',
 '23049590',
 '23049640',
 '23051998',
 '23091873',
 '23091874',
 '23092573',
 '23092574',
 '23092709']

In [22]:
len(need_id) # число статей, которые необходимо довыгрузить

42999

In [23]:
# если сложить эти недостающие 43 тысячи наблюдений и уже имеющиеся данные,
# то получится как раз 66473 наблюдений
42999+23474

66473

## Выгрузка. Часть 1.

В этой части производится попытка выгрузки недостающих 42 999 наблюдений. 

In [52]:
# обновляем df
df = df.iloc[0:0]
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref


In [54]:
counter = 0
for l in tqdm(range(len(need_id))): 
    line = need_id[l]
    code = line
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=026&itemid="+code
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    tree_response = urlopen(url).read()
    tree_data = ET.fromstring(tree_response)
    #print(ET.tostring(tree_data, encoding='utf8').decode('utf8')) #- вывести XML файл
    
    
    rus_auth_names = list()
    rus_auth_initials = list()
    rus_auth_affiliations = list()
    eng_auth_names = list()
    eng_auth_initials = list()
    eng_auth_affiliations = list()
    
    eng_auth_id=list()
    eng_afil_id=list()

    rus_auth_id=list()
    rus_afil_id=list()

    final_ids_ordered = list()
    final_auths_ordered = list()

    #### блок работы с авторами. 

    #xxx_auth_names - хранит в себе список фамилий авторов на языке xxx. 
    #xxx_auth_initials - хранит в себе список инициалов авторов на языке xxx. 
    #ххх_auth_affiliations - хранит в себе список списков аффилиаций авторов на языке xxx.
    #далее в df каждая ячейка тематики yyy будет выглядить так:
    #    [rus_auth_yyy],[eng_auth_yyy]
    try:
        author_data_for_numerizer = data['root']['item']['authors']['author']
        if type(author_data_for_numerizer) != list: #если всего один автор, то в XML он хранится в виде словаря а не списка словарей. поэтому, чтобы работало, делаем список словарей в любом случае
            author_data_for_numerizer = [author_data_for_numerizer]

        for j in range(1,int(max([x['@num'] for x in author_data_for_numerizer]))+1):


            author_data = list(filter(lambda z: z['@num'] == str(j), author_data_for_numerizer))

            rus_auth_names = rus_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_names = eng_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass


            rus_auth_initials = rus_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_initials = eng_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass

            try:
                rus_auth_affiliations = rus_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU',  author_data))]]]
                try:
                    eng_auth_affiliations = eng_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN',  author_data))]]]
                except:
                    pass
            except:
                pass
            
        #работа с id
            for kid in tree_data.iter('authors'):
                #print(tostring(kid, encoding='utf8').decode('utf8')) 
                for child in kid.iter('author'):


                    #print(tostring(child, encoding='utf8').decode('utf8'))

                    if child.attrib['lang'] == 'EN' and child.attrib['num'] == str(j):
                        eng_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                eng_afil_id.append(daughter.attrib['id'])
                        else:
                            eng_afil_id.append('none')

                    if child.attrib['lang'] == 'RU' and child.attrib['num'] == str(j):
                        rus_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                rus_afil_id.append(daughter.attrib['id'])
                        else:
                            rus_afil_id.append('none')

        while len(rus_afil_id)<9:
            rus_afil_id.append('none')
        while len(eng_afil_id)<9:
            eng_afil_id.append('none')

        while len(rus_auth_id)<9:
            rus_auth_id.append('none')
        while len(eng_auth_id)<9:
            eng_auth_id.append('none')

        for i in range(len(rus_afil_id)):
            if rus_afil_id[i]=='none':
                if eng_afil_id[i]!='none':
                    final_ids_ordered.append(eng_afil_id[i])
                else:
                    final_ids_ordered.append(rus_afil_id[i])
            else:
                final_ids_ordered.append(rus_afil_id[i])        

        for i in range(len(rus_auth_id)):
            if rus_auth_id[i]=='none':
                if eng_auth_id[i]!='none':
                    final_auths_ordered.append(eng_auth_id[i])
                else:
                    final_auths_ordered.append(rus_auth_id[i])
            else:
                final_auths_ordered.append(rus_auth_id[i])

        
        while len(rus_auth_names)<9:
            rus_auth_names.append('none')
            rus_auth_initials.append('none')
        while len(eng_auth_names)<9:
            eng_auth_names.append('none')
            eng_auth_initials.append('none')
        while len(rus_auth_affiliations)<9:
            rus_auth_affiliations.append('none')
        while len(eng_auth_affiliations)<9:
            eng_auth_affiliations.append('none')
        #while len(author_ids)<9:
        #    author_ids.append('none')
            
    except:
        final_ids_ordered = ['none' for x in range(9)]
        final_auths_ordered = ['none' for x in range(9)]
        rus_auth_names = ['none' for x in range(9)]
        rus_auth_initials = ['none' for x in range(9)]
        rus_auth_affiliations = ['none' for x in range(9)]
        eng_auth_names = ['none' for x in range(9)]
        eng_auth_initials = ['none' for x in range(9)]
        eng_auth_affiliations = ['none' for x in range(9)]


    
    
    #### блок работы с названиями работ.
    
    
    try:
        titles = data['root']['item']['titles']['title']
        if type(titles) != list:
            titles = [titles]             
        ru_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', titles))][0]
        try:
            eng_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', titles))][0]
        except:
            eng_title = 'none'
    except:
        ru_title = data['root']['item']['title']
        eng_title = 'none'
    
    
    #### блок работы с абстрактами
  
    
    try: #делаю траем, тк абстракта в XML может не быть
        abstract_data = data['root']['item']['abstracts']['abstract']

        
        ru_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'RU',  abstract_data))][0]
        try:
            eng_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'EN',  abstract_data))][0]
        except:
            eng_abstract = 'none'
    except:
        ru_abstract = 'none' 
        eng_abstract = 'none'
    
    
    #### блок работы с ключевыми словами

    
    try: #делаю траем, тк ключевых слов в XML может не быть
        keywords_data = data['root']['item']['keywords']['keyword']
        ru_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'RU', keywords_data))]
        try:
            eng_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'EN', keywords_data))]
        except:
            eng_keywords = 'none'
    except:
        ru_keywords = 'none'
        eng_keywords = 'none'
    if eng_keywords == []:
        eng_keywords = 'none'

    ### блок работы с кодами DOI и EDN
    code_data = data['root']['item']['itemCodes']['code']
    try:
        code_DOI = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'DOI', code_data))]
        try:
            code_EDN = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'EDN', code_data))]
            try: 
                code_UDK = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'УДК', code_data))]
            except:
                code_UDK = 'none'
        except:
            code_EDN = 'none'
    except:
        code_DOI = 'none'
        code_EDN = 'none'
        code_UDK = 'none'
    if code_DOI == []:
        code_DOI = 'none'
    if code_EDN == []:
        code_EDN = 'none'
    if code_UDK == []:
        code_UDK = 'none'
    
        
    
    ### блок работы с финансированием
    try: #делаю через try, тк информации о финансировании в XML может не быть
        fin_data = data['root']['item']['support']['supported']
        fin = fin_data['#text']#[x['#text'] for x in list(fin_data)]
        #print(fin)
    except:
        fin = None
    if fin == None:
        fin = 'none'
        
    ### блок работы с публикацией в журнале 
    try:
        number = data['root']['item']['number']
    except:
        number = 'none'
    try:
        jp = data['root']['item']['pages']
    except: 
        jp = 'none'
    try:
        np = data['root']['item']['pagesNumber']
    except:
        np = 'none'
    try: 
        volume = data['root']['item']['volume']
    except:
        volume = 'none'
    try:
        url = data['root']['item']['url']
    except:
        url = 'none'
        
    #### блок добавления новой строки
    
    
    new_row = {'id':data['root']['item']['@id'],
               'genreId':data['root']['item']['@genreId'],
             
               'author_1_id':final_auths_ordered[0], 
               'author_1_affiliation_id':final_ids_ordered[0],
               'author_1_ru last name':rus_auth_names[0], 
               'author_1_ru initials':rus_auth_initials[0],
               'author_1_ru affiliation':rus_auth_affiliations[0], 
               'author_1_eng last name':eng_auth_names[0], 
               'author_1_eng initials':eng_auth_initials[0],
               'author_1_eng affiliation':eng_auth_affiliations[0], 

               'author_2_id':final_auths_ordered[1], 
               'author_2_affiliation_id':final_ids_ordered[1],               
               'author_2_ru last name':rus_auth_names[1], 
               'author_2_ru initials':rus_auth_initials[1],
               'author_2_ru affiliation':rus_auth_affiliations[1], 
               'author_2_eng last name':eng_auth_names[1], 
               'author_2_eng initials':eng_auth_initials[1],
               'author_2_eng affiliation':eng_auth_affiliations[1], 

               'author_3_id':final_auths_ordered[2], 
               'author_3_affiliation_id':final_ids_ordered[2],               
               'author_3_ru last name':rus_auth_names[2], 
               'author_3_ru initials':rus_auth_initials[2],
               'author_3_ru affiliation':rus_auth_affiliations[2], 
               'author_3_eng last name':eng_auth_names[2], 
               'author_3_eng initials':eng_auth_initials[2],
               'author_3_eng affiliation':eng_auth_affiliations[2], 

               'author_4_id':final_auths_ordered[3], 
               'author_4_affiliation_id':final_ids_ordered[3],               
               'author_4_ru last name':rus_auth_names[3], 
               'author_4_ru initials':rus_auth_initials[3],
               'author_4_ru affiliation':rus_auth_affiliations[3], 
               'author_4_eng last name':eng_auth_names[3], 
               'author_4_eng initials':eng_auth_initials[3],
               'author_4_eng affiliation':eng_auth_affiliations[3], 

               'author_5_id':final_auths_ordered[4], 
               'author_5_affiliation_id':final_ids_ordered[4],           
               'author_5_ru last name':rus_auth_names[4], 
               'author_5_ru initials':rus_auth_initials[4],
               'author_5_ru affiliation':rus_auth_affiliations[4], 
               'author_5_eng last name':eng_auth_names[4], 
               'author_5_eng initials':eng_auth_initials[4],
               'author_5_eng affiliation':eng_auth_affiliations[4], 

               'author_6_id':final_auths_ordered[5], 
               'author_6_affiliation_id':final_ids_ordered[5],               
               'author_6_ru last name':rus_auth_names[5], 
               'author_6_ru initials':rus_auth_initials[5],
               'author_6_ru affiliation':rus_auth_affiliations[5], 
               'author_6_eng last name':eng_auth_names[5], 
               'author_6_eng initials':eng_auth_initials[5],
               'author_6_eng affiliation':eng_auth_affiliations[5], 

               'author_7_id':final_auths_ordered[6], 
               'author_7_affiliation_id':final_ids_ordered[6],               
               'author_7_ru last name':rus_auth_names[6], 
               'author_7_ru initials':rus_auth_initials[6],
               'author_7_ru affiliation':rus_auth_affiliations[6], 
               'author_7_eng last name':eng_auth_names[6], 
               'author_7_eng initials':eng_auth_initials[6],
               'author_7_eng affiliation':eng_auth_affiliations[6], 

               'author_8_id':final_auths_ordered[7], 
               'author_8_affiliation_id':final_ids_ordered[7],               
               'author_8_ru last name':rus_auth_names[7], 
               'author_8_ru initials':rus_auth_initials[7],
               'author_8_ru affiliation':rus_auth_affiliations[7], 
               'author_8_eng last name':eng_auth_names[7], 
               'author_8_eng initials':eng_auth_initials[7],
               'author_8_eng affiliation':eng_auth_affiliations[7], 
               
               'title_ru':ru_title, 
               'title_eng':eng_title, 
               
               'year':data['root']['item']['year'],
               
               'keywords_ru':ru_keywords,
               'keywords_eng':eng_keywords,
               
               'abstract_ru':ru_abstract,
               'abstract_eng':eng_abstract,
               
               'rubric':data['root']['item']['rubrics']['rubric']['#text'],
               'rubric_code':data['root']['item']['rubrics']['rubric']['@code'],
               'journal_name':data['root']['item']['journal']['#text'],
               'journal_id':data['root']['item']['journal']['@id'],
               'journal_issn':data['root']['item']['journal']['@issn'],
               'journal_eissn':data['root']['item']['journal']['@eissn'],
               'journnal_impact_factor':data['root']['item']['journal']['@impactFactor'],
               'journal_VAK':data['root']['item']['journal']['@VAK'],
               'journal_Scopus':data['root']['item']['journal']['@Scopus'],
               'journal_WebofScience':data['root']['item']['journal']['@WebofScience'],
               'journal_RSCI':data['root']['item']['journal']['@RSCI'],
               'journal_RISC':data['root']['item']['journal']['@RISC'],
               'volume':volume,
               'journal_number':number,
               'journal_pages':jp,
               'number_of_pages':np,
               'publisher_id':data['root']['item']['publisher']['@id'],
               'publisher':data['root']['item']['publisher']['#text'],
               'DOI':code_DOI,
               'EDN':code_EDN,
               'UDK':code_UDK,
               'support': fin,
                               
               'genre_id':data['root']['item']['@genreId'],
               'cited':data['root']['item']['@cited'],
               'corecited':data['root']['item']['@coreCited'],
               'parent_id':data['root']['item']['@parentId'],
               'source_id':data['root']['item']['@sourceId'],
               'isFT':data['root']['item']['@isFT'],
               'isNEW':data['root']['item']['@isNew'],
               'link':data['root']['item']['link'],
               'url':url,
               'ref':data['root']['item']['ref']
              }
    
    
    df = df.append(new_row, ignore_index=True)
    counter += 1
    
    # каждые 50 строк засыпаем на 2 секунды чтобы не перегрузить сервис
    if counter % 110 == 0:
        sleep(1.8)
    # каждые 1000 строк сохраняем промежуточную выгрузку
    if counter % 1000 == 0:
        df.to_csv(f'df_7_july_{counter}_rows.csv')
        df.to_excel(f'df_7_july_{counter}_rows.xlsx')

 57%|█████▋    | 24620/42999 [2:33:30<1:54:35,  2.67it/s]  


KeyError: 'publisher'

Выгрузка данных остановилась на середине из-за ошибки. Было выгружено 24 620 наблюдений. 

In [55]:
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,35534234,4,331427,1068,Ильиных,Светлана Анатольевна,Новосибирский государственный университет экон...,Ilyinykh,Svetlana Anatolyevna,Novosibirsk State University of Economics and ...,...,4,0,0,35534233,,1,0,http://elibrary.ru/item.asp?id=35534234,none,"Ильиных, С.А. Идеологические и кадровые инстру..."
24616,35534241,4,833623,7082,Соколова,Марина Владимировна,Владимирский филиал Российской академии народн...,Sokolova,Marina Vladimirovna,Vladimir branch of Russian Presidential Academ...,...,4,1,0,35534233,,1,0,http://elibrary.ru/item.asp?id=35534241,none,"Соколова, М.В. Социально-трудовая мобильность ..."
24617,35546023,4,284007,7964,Быченко,Юрий Григорьевич,Саратовский военный ордена Жукова Краснознамен...,Bychenko,Y.G.,Saratov military order of Zhukov red banner In...,...,4,14,1,35546019,,1,0,http://elibrary.ru/item.asp?id=35546023,none,"Быченко, Ю.Г. СОВЕРШЕНСТВОВАНИЕ МОТИВАЦИИ СЛУЖ..."
24618,35551539,4,811101,369,Кныш,Наталья Александровна,Омский государственный университет,Knysh,Natal'ya Aleksandrovna,Omsk State University,...,4,1,0,35551537,,1,0,http://elibrary.ru/item.asp?id=35551539,none,"Knysh, N.A. The transformation of the image of..."


Сохраняем эти наблюдения в файл `df_7_july_24620_rows`, один - с расширением .csv, другой - с расширением .xlsx.

In [57]:
df.to_excel('df_7_july_24620_rows.xlsx')
df.to_csv('df_7_july_24620_rows.csv')

## Выгрузка. Часть 2.

Продолжаю выгрузку с 24621 элемента, на котором остановился процесс.

In [62]:
counter = 24620
for l in tqdm(range(len(need_id[24621:]))): 
    line = need_id[l]
    code = line
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=026&itemid="+code
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    tree_response = urlopen(url).read()
    tree_data = ET.fromstring(tree_response)
    #print(ET.tostring(tree_data, encoding='utf8').decode('utf8')) #- вывести XML файл
    
    
    rus_auth_names = list()
    rus_auth_initials = list()
    rus_auth_affiliations = list()
    eng_auth_names = list()
    eng_auth_initials = list()
    eng_auth_affiliations = list()
    
    eng_auth_id=list()
    eng_afil_id=list()

    rus_auth_id=list()
    rus_afil_id=list()

    final_ids_ordered = list()
    final_auths_ordered = list()

    #### блок работы с авторами. 

    #xxx_auth_names - хранит в себе список фамилий авторов на языке xxx. 
    #xxx_auth_initials - хранит в себе список инициалов авторов на языке xxx. 
    #ххх_auth_affiliations - хранит в себе список списков аффилиаций авторов на языке xxx.
    #далее в df каждая ячейка тематики yyy будет выглядить так:
    #    [rus_auth_yyy],[eng_auth_yyy]
    try:
        author_data_for_numerizer = data['root']['item']['authors']['author']
        if type(author_data_for_numerizer) != list: #если всего один автор, то в XML он хранится в виде словаря а не списка словарей. поэтому, чтобы работало, делаем список словарей в любом случае
            author_data_for_numerizer = [author_data_for_numerizer]

        for j in range(1,int(max([x['@num'] for x in author_data_for_numerizer]))+1):


            author_data = list(filter(lambda z: z['@num'] == str(j), author_data_for_numerizer))

            rus_auth_names = rus_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_names = eng_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass


            rus_auth_initials = rus_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_initials = eng_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass

            try:
                rus_auth_affiliations = rus_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU',  author_data))]]]
                try:
                    eng_auth_affiliations = eng_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN',  author_data))]]]
                except:
                    pass
            except:
                pass
            
        #работа с id
            for kid in tree_data.iter('authors'):
                #print(tostring(kid, encoding='utf8').decode('utf8')) 
                for child in kid.iter('author'):


                    #print(tostring(child, encoding='utf8').decode('utf8'))

                    if child.attrib['lang'] == 'EN' and child.attrib['num'] == str(j):
                        eng_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                eng_afil_id.append(daughter.attrib['id'])
                        else:
                            eng_afil_id.append('none')

                    if child.attrib['lang'] == 'RU' and child.attrib['num'] == str(j):
                        rus_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                rus_afil_id.append(daughter.attrib['id'])
                        else:
                            rus_afil_id.append('none')

        while len(rus_afil_id)<9:
            rus_afil_id.append('none')
        while len(eng_afil_id)<9:
            eng_afil_id.append('none')

        while len(rus_auth_id)<9:
            rus_auth_id.append('none')
        while len(eng_auth_id)<9:
            eng_auth_id.append('none')

        for i in range(len(rus_afil_id)):
            if rus_afil_id[i]=='none':
                if eng_afil_id[i]!='none':
                    final_ids_ordered.append(eng_afil_id[i])
                else:
                    final_ids_ordered.append(rus_afil_id[i])
            else:
                final_ids_ordered.append(rus_afil_id[i])        

        for i in range(len(rus_auth_id)):
            if rus_auth_id[i]=='none':
                if eng_auth_id[i]!='none':
                    final_auths_ordered.append(eng_auth_id[i])
                else:
                    final_auths_ordered.append(rus_auth_id[i])
            else:
                final_auths_ordered.append(rus_auth_id[i])

        
        while len(rus_auth_names)<9:
            rus_auth_names.append('none')
            rus_auth_initials.append('none')
        while len(eng_auth_names)<9:
            eng_auth_names.append('none')
            eng_auth_initials.append('none')
        while len(rus_auth_affiliations)<9:
            rus_auth_affiliations.append('none')
        while len(eng_auth_affiliations)<9:
            eng_auth_affiliations.append('none')
        #while len(author_ids)<9:
        #    author_ids.append('none')
            
    except:
        final_ids_ordered = ['none' for x in range(9)]
        final_auths_ordered = ['none' for x in range(9)]
        rus_auth_names = ['none' for x in range(9)]
        rus_auth_initials = ['none' for x in range(9)]
        rus_auth_affiliations = ['none' for x in range(9)]
        eng_auth_names = ['none' for x in range(9)]
        eng_auth_initials = ['none' for x in range(9)]
        eng_auth_affiliations = ['none' for x in range(9)]


    
    
    #### блок работы с названиями работ.
    
    
    try:
        titles = data['root']['item']['titles']['title']
        if type(titles) != list:
            titles = [titles]             
        ru_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', titles))][0]
        try:
            eng_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', titles))][0]
        except:
            eng_title = 'none'
    except:
        ru_title = data['root']['item']['title']
        eng_title = 'none'
    
    
    #### блок работы с абстрактами
  
    
    try: #делаю траем, тк абстракта в XML может не быть
        abstract_data = data['root']['item']['abstracts']['abstract']

        
        ru_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'RU',  abstract_data))][0]
        try:
            eng_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'EN',  abstract_data))][0]
        except:
            eng_abstract = 'none'
    except:
        ru_abstract = 'none' 
        eng_abstract = 'none'
    
    
    #### блок работы с ключевыми словами

    
    try: #делаю траем, тк ключевых слов в XML может не быть
        keywords_data = data['root']['item']['keywords']['keyword']
        ru_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'RU', keywords_data))]
        try:
            eng_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'EN', keywords_data))]
        except:
            eng_keywords = 'none'
    except:
        ru_keywords = 'none'
        eng_keywords = 'none'
    if eng_keywords == []:
        eng_keywords = 'none'

    ### блок работы с кодами DOI и EDN
    code_data = data['root']['item']['itemCodes']['code']
    try:
        code_DOI = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'DOI', code_data))]
        try:
            code_EDN = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'EDN', code_data))]
            try: 
                code_UDK = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'УДК', code_data))]
            except:
                code_UDK = 'none'
        except:
            code_EDN = 'none'
    except:
        code_DOI = 'none'
        code_EDN = 'none'
        code_UDK = 'none'
    if code_DOI == []:
        code_DOI = 'none'
    if code_EDN == []:
        code_EDN = 'none'
    if code_UDK == []:
        code_UDK = 'none'
    
        
    
    ### блок работы с финансированием
    try: #делаю через try, тк информации о финансировании в XML может не быть
        fin_data = data['root']['item']['support']['supported']
        fin = fin_data['#text']#[x['#text'] for x in list(fin_data)]
        #print(fin)
    except:
        fin = None
    if fin == None:
        fin = 'none'
        
    ### блок работы с публикацией в журнале 
    try:
        number = data['root']['item']['number']
    except:
        number = 'none'
    try:
        jp = data['root']['item']['pages']
    except: 
        jp = 'none'
    try:
        np = data['root']['item']['pagesNumber']
    except:
        np = 'none'
    try: 
        volume = data['root']['item']['volume']
    except:
        volume = 'none'
    try:
        url = data['root']['item']['url']
    except:
        url = 'none'
        
    #### блок добавления новой строки
    
    
    new_row = {'id':data['root']['item']['@id'],
               'genreId':data['root']['item']['@genreId'],
             
               'author_1_id':final_auths_ordered[0], 
               'author_1_affiliation_id':final_ids_ordered[0],
               'author_1_ru last name':rus_auth_names[0], 
               'author_1_ru initials':rus_auth_initials[0],
               'author_1_ru affiliation':rus_auth_affiliations[0], 
               'author_1_eng last name':eng_auth_names[0], 
               'author_1_eng initials':eng_auth_initials[0],
               'author_1_eng affiliation':eng_auth_affiliations[0], 

               'author_2_id':final_auths_ordered[1], 
               'author_2_affiliation_id':final_ids_ordered[1],               
               'author_2_ru last name':rus_auth_names[1], 
               'author_2_ru initials':rus_auth_initials[1],
               'author_2_ru affiliation':rus_auth_affiliations[1], 
               'author_2_eng last name':eng_auth_names[1], 
               'author_2_eng initials':eng_auth_initials[1],
               'author_2_eng affiliation':eng_auth_affiliations[1], 

               'author_3_id':final_auths_ordered[2], 
               'author_3_affiliation_id':final_ids_ordered[2],               
               'author_3_ru last name':rus_auth_names[2], 
               'author_3_ru initials':rus_auth_initials[2],
               'author_3_ru affiliation':rus_auth_affiliations[2], 
               'author_3_eng last name':eng_auth_names[2], 
               'author_3_eng initials':eng_auth_initials[2],
               'author_3_eng affiliation':eng_auth_affiliations[2], 

               'author_4_id':final_auths_ordered[3], 
               'author_4_affiliation_id':final_ids_ordered[3],               
               'author_4_ru last name':rus_auth_names[3], 
               'author_4_ru initials':rus_auth_initials[3],
               'author_4_ru affiliation':rus_auth_affiliations[3], 
               'author_4_eng last name':eng_auth_names[3], 
               'author_4_eng initials':eng_auth_initials[3],
               'author_4_eng affiliation':eng_auth_affiliations[3], 

               'author_5_id':final_auths_ordered[4], 
               'author_5_affiliation_id':final_ids_ordered[4],           
               'author_5_ru last name':rus_auth_names[4], 
               'author_5_ru initials':rus_auth_initials[4],
               'author_5_ru affiliation':rus_auth_affiliations[4], 
               'author_5_eng last name':eng_auth_names[4], 
               'author_5_eng initials':eng_auth_initials[4],
               'author_5_eng affiliation':eng_auth_affiliations[4], 

               'author_6_id':final_auths_ordered[5], 
               'author_6_affiliation_id':final_ids_ordered[5],               
               'author_6_ru last name':rus_auth_names[5], 
               'author_6_ru initials':rus_auth_initials[5],
               'author_6_ru affiliation':rus_auth_affiliations[5], 
               'author_6_eng last name':eng_auth_names[5], 
               'author_6_eng initials':eng_auth_initials[5],
               'author_6_eng affiliation':eng_auth_affiliations[5], 

               'author_7_id':final_auths_ordered[6], 
               'author_7_affiliation_id':final_ids_ordered[6],               
               'author_7_ru last name':rus_auth_names[6], 
               'author_7_ru initials':rus_auth_initials[6],
               'author_7_ru affiliation':rus_auth_affiliations[6], 
               'author_7_eng last name':eng_auth_names[6], 
               'author_7_eng initials':eng_auth_initials[6],
               'author_7_eng affiliation':eng_auth_affiliations[6], 

               'author_8_id':final_auths_ordered[7], 
               'author_8_affiliation_id':final_ids_ordered[7],               
               'author_8_ru last name':rus_auth_names[7], 
               'author_8_ru initials':rus_auth_initials[7],
               'author_8_ru affiliation':rus_auth_affiliations[7], 
               'author_8_eng last name':eng_auth_names[7], 
               'author_8_eng initials':eng_auth_initials[7],
               'author_8_eng affiliation':eng_auth_affiliations[7], 
               
               'title_ru':ru_title, 
               'title_eng':eng_title, 
               
               'year':data['root']['item']['year'],
               
               'keywords_ru':ru_keywords,
               'keywords_eng':eng_keywords,
               
               'abstract_ru':ru_abstract,
               'abstract_eng':eng_abstract,
               
               'rubric':data['root']['item']['rubrics']['rubric']['#text'],
               'rubric_code':data['root']['item']['rubrics']['rubric']['@code'],
               'journal_name':data['root']['item']['journal']['#text'],
               'journal_id':data['root']['item']['journal']['@id'],
               'journal_issn':data['root']['item']['journal']['@issn'],
               'journal_eissn':data['root']['item']['journal']['@eissn'],
               'journnal_impact_factor':data['root']['item']['journal']['@impactFactor'],
               'journal_VAK':data['root']['item']['journal']['@VAK'],
               'journal_Scopus':data['root']['item']['journal']['@Scopus'],
               'journal_WebofScience':data['root']['item']['journal']['@WebofScience'],
               'journal_RSCI':data['root']['item']['journal']['@RSCI'],
               'journal_RISC':data['root']['item']['journal']['@RISC'],
               'volume':volume,
               'journal_number':number,
               'journal_pages':jp,
               'number_of_pages':np,
               'publisher_id':data['root']['item']['publisher']['@id'],
               'publisher':data['root']['item']['publisher']['#text'],
               'DOI':code_DOI,
               'EDN':code_EDN,
               'UDK':code_UDK,
               'support': fin,
                               
               'genre_id':data['root']['item']['@genreId'],
               'cited':data['root']['item']['@cited'],
               'corecited':data['root']['item']['@coreCited'],
               'parent_id':data['root']['item']['@parentId'],
               'source_id':data['root']['item']['@sourceId'],
               'isFT':data['root']['item']['@isFT'],
               'isNEW':data['root']['item']['@isNew'],
               'link':data['root']['item']['link'],
               'url':url,
               'ref':data['root']['item']['ref']
              }
    
    
    df = df.append(new_row, ignore_index=True)
    counter += 1
    
    # каждые 50 строк засыпаем на 2 секунды чтобы не перегрузить сервис
    if counter % 110 == 0:
        sleep(1.8)
    # каждые 1000 строк сохраняем промежуточную выгрузку
    if counter % 1000 == 0:
        df.to_csv(f'df_7_july_{counter}_rows.csv')
        df.to_excel(f'df_7_july_{counter}_rows.xlsx')

100%|██████████| 18378/18378 [2:20:58<00:00,  2.17it/s]    


In [64]:
df.to_csv('df_7_july_42998_rows.csv')
df.to_excel('df_7_july_42998_rows.xlsx')

Смотрим на получившийся набор данных.

In [65]:
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42993,35111403,4,0,953,Исичко,Анастасия Олеговна,Самарский университет,Isichko,Anastasia Olegovna,Samara University,...,4,0,0,35111397,,1,0,http://elibrary.ru/item.asp?id=35111403,none,"Исичко, А.О. ОБЩЕСТВО ПОТРЕБЛЕНИЯ И ЕГО АЛЬТЕР..."
42994,35119978,4,391667,258,Нагайцев,Виктор Валентинович,Алтайский государственный университет,Nagaitsev,Victor V.,Altai State University,...,4,0,0,35119959,,1,0,http://elibrary.ru/item.asp?id=35119978,none,"Нагайцев, В.В. Основные стратегии поведения на..."
42995,34996304,4,0,258,Самойлов,Н.Ю.,Алтайский государственный университет,none,none,none,...,4,1,0,34996279,,1,1,http://elibrary.ru/item.asp?id=34996304,None,"Самойлов, Н.Ю. МОЛОДЕЖНЫЕ СУБКУЛЬТУРЫ ВЕЛИКОБР..."
42996,34996355,4,0,258,Хатунцева,П.Е.,Алтайский государственный университет,none,none,none,...,4,0,0,34996279,,1,1,http://elibrary.ru/item.asp?id=34996355,None,"Хатунцева, П.Е. КРИТЕРИИ ПРОГРЕССА И РЕГРЕССА ..."


На этом этапе выяснилось, что из-за технической ошибки с нашей стороны, данные выгружались некорректно и у нас есть только 24 620 наблюдений.

In [66]:
df.drop_duplicates('id')

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,35534234,4,331427,1068,Ильиных,Светлана Анатольевна,Новосибирский государственный университет экон...,Ilyinykh,Svetlana Anatolyevna,Novosibirsk State University of Economics and ...,...,4,0,0,35534233,,1,0,http://elibrary.ru/item.asp?id=35534234,none,"Ильиных, С.А. Идеологические и кадровые инстру..."
24616,35534241,4,833623,7082,Соколова,Марина Владимировна,Владимирский филиал Российской академии народн...,Sokolova,Marina Vladimirovna,Vladimir branch of Russian Presidential Academ...,...,4,1,0,35534233,,1,0,http://elibrary.ru/item.asp?id=35534241,none,"Соколова, М.В. Социально-трудовая мобильность ..."
24617,35546023,4,284007,7964,Быченко,Юрий Григорьевич,Саратовский военный ордена Жукова Краснознамен...,Bychenko,Y.G.,Saratov military order of Zhukov red banner In...,...,4,14,1,35546019,,1,0,http://elibrary.ru/item.asp?id=35546023,none,"Быченко, Ю.Г. СОВЕРШЕНСТВОВАНИЕ МОТИВАЦИИ СЛУЖ..."
24618,35551539,4,811101,369,Кныш,Наталья Александровна,Омский государственный университет,Knysh,Natal'ya Aleksandrovna,Omsk State University,...,4,1,0,35551537,,1,0,http://elibrary.ru/item.asp?id=35551539,none,"Knysh, N.A. The transformation of the image of..."


Для дальнейшей работы мы объединяем уже имеющийся датафрейм и оформленную незадолго до этого выгрузку (`first_part`).

In [67]:
try_to_unite = pd.concat([df.drop_duplicates('id'), first_part])
try_to_unite

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38569,23018570,4,811563,305,Давлетшина,Гульнара Рафисовна,ФГБОУ ВПО «Башкирский государственный медицинс...,none,none,none,...,4,3,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018570,NaN,"Давлетшина, Г.Р. ТРУДНОСТИ АДАПТАЦИИ ИНДИВИДОВ..."
38570,23018587,4,415461,804,Прокшина,Лира Рифгатовна,Башкирский институт социальных технологий (фил...,none,none,none,...,4,0,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018587,NaN,"Прокшина, Л.Р. ИГРА КАК АДАПТАЦИОННЫЙ ПОТЕНЦИА..."
38571,23025275,4,0,211,Гареева,Лена Фларитовна,Башкирский государственный университет,Gareeva,L.F.,none,...,4,0,0,34054658,NaN,1,0,http://elibrary.ru/item.asp?id=23025275,none,"Гареева, Л.Ф. Социально-философские проблемы с..."
38572,23027380,4,215177,7082,Васильева,Е.И.,Российская академия народного хозяйства и госу...,Vasilyeva,E.I.,Russian Presidential Academy of National Econo...,...,4,0,0,34054790,NaN,0,0,http://elibrary.ru/item.asp?id=23027380,none,"Васильева, Е.И. Качество и доступность предост..."


Видим, что по имеющимся данным – у нас есть 48 094 наблюдения

In [71]:
try_to_unite.drop_duplicates('id')

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38569,23018570,4,811563,305,Давлетшина,Гульнара Рафисовна,ФГБОУ ВПО «Башкирский государственный медицинс...,none,none,none,...,4,3,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018570,NaN,"Давлетшина, Г.Р. ТРУДНОСТИ АДАПТАЦИИ ИНДИВИДОВ..."
38570,23018587,4,415461,804,Прокшина,Лира Рифгатовна,Башкирский институт социальных технологий (фил...,none,none,none,...,4,0,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018587,NaN,"Прокшина, Л.Р. ИГРА КАК АДАПТАЦИОННЫЙ ПОТЕНЦИА..."
38571,23025275,4,0,211,Гареева,Лена Фларитовна,Башкирский государственный университет,Gareeva,L.F.,none,...,4,0,0,34054658,NaN,1,0,http://elibrary.ru/item.asp?id=23025275,none,"Гареева, Л.Ф. Социально-философские проблемы с..."
38572,23027380,4,215177,7082,Васильева,Е.И.,Российская академия народного хозяйства и госу...,Vasilyeva,E.I.,Russian Presidential Academy of National Econo...,...,4,0,0,34054790,NaN,0,0,http://elibrary.ru/item.asp?id=23027380,none,"Васильева, Е.И. Качество и доступность предост..."


Сохраняем эти данные в формате .csv и .xlsx.

In [70]:
try_to_unite.to_excel('df_7_july_48094_rows.xlsx')
try_to_unite.to_csv('df_7_july_48094_rows.csv')

## Выгрузка. Часть 3.

Для того чтобы облегчить следующую работу, переприсваиваю df - там находится файл с 48 094 строчками, полученный ранее.

In [72]:
df = try_to_unite
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38569,23018570,4,811563,305,Давлетшина,Гульнара Рафисовна,ФГБОУ ВПО «Башкирский государственный медицинс...,none,none,none,...,4,3,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018570,NaN,"Давлетшина, Г.Р. ТРУДНОСТИ АДАПТАЦИИ ИНДИВИДОВ..."
38570,23018587,4,415461,804,Прокшина,Лира Рифгатовна,Башкирский институт социальных технологий (фил...,none,none,none,...,4,0,0,34054341,NaN,1,1,http://elibrary.ru/item.asp?id=23018587,NaN,"Прокшина, Л.Р. ИГРА КАК АДАПТАЦИОННЫЙ ПОТЕНЦИА..."
38571,23025275,4,0,211,Гареева,Лена Фларитовна,Башкирский государственный университет,Gareeva,L.F.,none,...,4,0,0,34054658,NaN,1,0,http://elibrary.ru/item.asp?id=23025275,none,"Гареева, Л.Ф. Социально-философские проблемы с..."
38572,23027380,4,215177,7082,Васильева,Е.И.,Российская академия народного хозяйства и госу...,Vasilyeva,E.I.,Russian Presidential Academy of National Econo...,...,4,0,0,34054790,NaN,0,0,http://elibrary.ru/item.asp?id=23027380,none,"Васильева, Е.И. Качество и доступность предост..."


В список `last_part` записываю те id статей, которые не находятся в нашем датафрейме.

In [73]:
last_part = [i for i in need_id if i not in list(df['id'])]
last_part[0:10]

['35554604',
 '35554804',
 '35554808',
 '35555708',
 '35559370',
 '35559863',
 '35426781',
 '35427113',
 '35447880',
 '35449302']

Смотрю на число таких id.

In [74]:
len(last_part)

18379

Действительно, если довыгрузить эти данные и объединить их с уже имеющимся датафреймом - получится 66 473 наблюдения.

In [75]:
len(last_part) + df.shape[0] == 66473

True

In [88]:
df = df.drop_duplicates('id')
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48105,35555708,4,639254,,Семенова,Виктория Владимировна,none,Semenova,Victoria Vladimirovna,none,...,4,0,0,35555707,,1,0,http://elibrary.ru/item.asp?id=35555708,none,"Семенова, В.В. Письмо редакторов / В.В. Семено..."
48106,35559370,4,0,2541,Шаров,Константин Сергеевич,МГУ имени М. В. Ломоносова,none,none,none,...,4,0,0,35559352,,0,1,http://elibrary.ru/item.asp?id=35559370,None,"Шаров, К.С. Ньютоновское исследование физико-т..."
48107,35559863,4,1110181,7082,Тирабян,Карине Камоевна,Северо-Западный институт управления РАНХиГС,Tirabian,Karine K.,North-West institute of management of RANEPA,...,4,0,0,35559844,,1,0,http://elibrary.ru/item.asp?id=35559863,none,"Тирабян, К.К. Роль религии при формировании ар..."
48108,35426781,4,658394,13718,Бондарев,Виктор Геннадьевич,Северо-Западный филиал Российского государстве...,Bondarev,Viktor Gennadyevich,North-Western branch of Russian State Universi...,...,4,2,0,35426779,,1,0,http://elibrary.ru/item.asp?id=35426781,none,"Бондарев, В.Г. Справедливость правосудия как с..."


Произвожу выгрузку

In [89]:
counter = 48094
for l in tqdm(range(len(last_part))): 
    line = last_part[l]
    code = line
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=026&itemid="+code
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    tree_response = urlopen(url).read()
    tree_data = ET.fromstring(tree_response)
    #print(ET.tostring(tree_data, encoding='utf8').decode('utf8')) #- вывести XML файл
    
    
    rus_auth_names = list()
    rus_auth_initials = list()
    rus_auth_affiliations = list()
    eng_auth_names = list()
    eng_auth_initials = list()
    eng_auth_affiliations = list()
    
    eng_auth_id=list()
    eng_afil_id=list()

    rus_auth_id=list()
    rus_afil_id=list()

    final_ids_ordered = list()
    final_auths_ordered = list()

    #### блок работы с авторами. 

    #xxx_auth_names - хранит в себе список фамилий авторов на языке xxx. 
    #xxx_auth_initials - хранит в себе список инициалов авторов на языке xxx. 
    #ххх_auth_affiliations - хранит в себе список списков аффилиаций авторов на языке xxx.
    #далее в df каждая ячейка тематики yyy будет выглядить так:
    #    [rus_auth_yyy],[eng_auth_yyy]
    try:
        author_data_for_numerizer = data['root']['item']['authors']['author']
        if type(author_data_for_numerizer) != list: #если всего один автор, то в XML он хранится в виде словаря а не списка словарей. поэтому, чтобы работало, делаем список словарей в любом случае
            author_data_for_numerizer = [author_data_for_numerizer]

        for j in range(1,int(max([x['@num'] for x in author_data_for_numerizer]))+1):


            author_data = list(filter(lambda z: z['@num'] == str(j), author_data_for_numerizer))

            rus_auth_names = rus_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_names = eng_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass


            rus_auth_initials = rus_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_initials = eng_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass

            try:
                rus_auth_affiliations = rus_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU',  author_data))]]]
                try:
                    eng_auth_affiliations = eng_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN',  author_data))]]]
                except:
                    pass
            except:
                pass
            
        #работа с id
            for kid in tree_data.iter('authors'):
                #print(tostring(kid, encoding='utf8').decode('utf8')) 
                for child in kid.iter('author'):


                    #print(tostring(child, encoding='utf8').decode('utf8'))

                    if child.attrib['lang'] == 'EN' and child.attrib['num'] == str(j):
                        eng_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                eng_afil_id.append(daughter.attrib['id'])
                        else:
                            eng_afil_id.append('none')

                    if child.attrib['lang'] == 'RU' and child.attrib['num'] == str(j):
                        rus_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                rus_afil_id.append(daughter.attrib['id'])
                        else:
                            rus_afil_id.append('none')

        while len(rus_afil_id)<9:
            rus_afil_id.append('none')
        while len(eng_afil_id)<9:
            eng_afil_id.append('none')

        while len(rus_auth_id)<9:
            rus_auth_id.append('none')
        while len(eng_auth_id)<9:
            eng_auth_id.append('none')

        for i in range(len(rus_afil_id)):
            if rus_afil_id[i]=='none':
                if eng_afil_id[i]!='none':
                    final_ids_ordered.append(eng_afil_id[i])
                else:
                    final_ids_ordered.append(rus_afil_id[i])
            else:
                final_ids_ordered.append(rus_afil_id[i])        

        for i in range(len(rus_auth_id)):
            if rus_auth_id[i]=='none':
                if eng_auth_id[i]!='none':
                    final_auths_ordered.append(eng_auth_id[i])
                else:
                    final_auths_ordered.append(rus_auth_id[i])
            else:
                final_auths_ordered.append(rus_auth_id[i])

        
        while len(rus_auth_names)<9:
            rus_auth_names.append('none')
            rus_auth_initials.append('none')
        while len(eng_auth_names)<9:
            eng_auth_names.append('none')
            eng_auth_initials.append('none')
        while len(rus_auth_affiliations)<9:
            rus_auth_affiliations.append('none')
        while len(eng_auth_affiliations)<9:
            eng_auth_affiliations.append('none')
        #while len(author_ids)<9:
        #    author_ids.append('none')
            
    except:
        final_ids_ordered = ['none' for x in range(9)]
        final_auths_ordered = ['none' for x in range(9)]
        rus_auth_names = ['none' for x in range(9)]
        rus_auth_initials = ['none' for x in range(9)]
        rus_auth_affiliations = ['none' for x in range(9)]
        eng_auth_names = ['none' for x in range(9)]
        eng_auth_initials = ['none' for x in range(9)]
        eng_auth_affiliations = ['none' for x in range(9)]


    
    
    #### блок работы с названиями работ.
    
    
    try:
        titles = data['root']['item']['titles']['title']
        if type(titles) != list:
            titles = [titles]             
        ru_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', titles))][0]
        try:
            eng_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', titles))][0]
        except:
            eng_title = 'none'
    except:
        ru_title = data['root']['item']['title']
        eng_title = 'none'
    
    
    #### блок работы с абстрактами
  
    
    try: #делаю траем, тк абстракта в XML может не быть
        abstract_data = data['root']['item']['abstracts']['abstract']

        
        ru_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'RU',  abstract_data))][0]
        try:
            eng_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'EN',  abstract_data))][0]
        except:
            eng_abstract = 'none'
    except:
        ru_abstract = 'none' 
        eng_abstract = 'none'
    
    
    #### блок работы с ключевыми словами

    
    try: #делаю траем, тк ключевых слов в XML может не быть
        keywords_data = data['root']['item']['keywords']['keyword']
        ru_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'RU', keywords_data))]
        try:
            eng_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'EN', keywords_data))]
        except:
            eng_keywords = 'none'
    except:
        ru_keywords = 'none'
        eng_keywords = 'none'
    if eng_keywords == []:
        eng_keywords = 'none'

    ### блок работы с кодами DOI и EDN
    code_data = data['root']['item']['itemCodes']['code']
    try:
        code_DOI = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'DOI', code_data))]
        try:
            code_EDN = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'EDN', code_data))]
            try: 
                code_UDK = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'УДК', code_data))]
            except:
                code_UDK = 'none'
        except:
            code_EDN = 'none'
    except:
        code_DOI = 'none'
        code_EDN = 'none'
        code_UDK = 'none'
    if code_DOI == []:
        code_DOI = 'none'
    if code_EDN == []:
        code_EDN = 'none'
    if code_UDK == []:
        code_UDK = 'none'
    
        
    
    ### блок работы с финансированием
    try: #делаю через try, тк информации о финансировании в XML может не быть
        fin_data = data['root']['item']['support']['supported']
        fin = fin_data['#text']#[x['#text'] for x in list(fin_data)]
        #print(fin)
    except:
        fin = None
    if fin == None:
        fin = 'none'
        
    ### блок работы с публикацией в журнале 
    try:
        number = data['root']['item']['number']
    except:
        number = 'none'
    try:
        jp = data['root']['item']['pages']
    except: 
        jp = 'none'
    try:
        np = data['root']['item']['pagesNumber']
    except:
        np = 'none'
    try: 
        volume = data['root']['item']['volume']
    except:
        volume = 'none'
    try:
        url = data['root']['item']['url']
    except:
        url = 'none'
        
    try:
        pub_data = data['root']['item']['publisher']
        publisher = pub_data['#text']
        publisher_id = publisher_id['@id']
    except:
        publisher = 'none'
        publisher_id = 'none'
    #### блок добавления новой строки
    
    
    new_row = {'id':data['root']['item']['@id'],
               'genreId':data['root']['item']['@genreId'],
             
               'author_1_id':final_auths_ordered[0], 
               'author_1_affiliation_id':final_ids_ordered[0],
               'author_1_ru last name':rus_auth_names[0], 
               'author_1_ru initials':rus_auth_initials[0],
               'author_1_ru affiliation':rus_auth_affiliations[0], 
               'author_1_eng last name':eng_auth_names[0], 
               'author_1_eng initials':eng_auth_initials[0],
               'author_1_eng affiliation':eng_auth_affiliations[0], 

               'author_2_id':final_auths_ordered[1], 
               'author_2_affiliation_id':final_ids_ordered[1],               
               'author_2_ru last name':rus_auth_names[1], 
               'author_2_ru initials':rus_auth_initials[1],
               'author_2_ru affiliation':rus_auth_affiliations[1], 
               'author_2_eng last name':eng_auth_names[1], 
               'author_2_eng initials':eng_auth_initials[1],
               'author_2_eng affiliation':eng_auth_affiliations[1], 

               'author_3_id':final_auths_ordered[2], 
               'author_3_affiliation_id':final_ids_ordered[2],               
               'author_3_ru last name':rus_auth_names[2], 
               'author_3_ru initials':rus_auth_initials[2],
               'author_3_ru affiliation':rus_auth_affiliations[2], 
               'author_3_eng last name':eng_auth_names[2], 
               'author_3_eng initials':eng_auth_initials[2],
               'author_3_eng affiliation':eng_auth_affiliations[2], 

               'author_4_id':final_auths_ordered[3], 
               'author_4_affiliation_id':final_ids_ordered[3],               
               'author_4_ru last name':rus_auth_names[3], 
               'author_4_ru initials':rus_auth_initials[3],
               'author_4_ru affiliation':rus_auth_affiliations[3], 
               'author_4_eng last name':eng_auth_names[3], 
               'author_4_eng initials':eng_auth_initials[3],
               'author_4_eng affiliation':eng_auth_affiliations[3], 

               'author_5_id':final_auths_ordered[4], 
               'author_5_affiliation_id':final_ids_ordered[4],           
               'author_5_ru last name':rus_auth_names[4], 
               'author_5_ru initials':rus_auth_initials[4],
               'author_5_ru affiliation':rus_auth_affiliations[4], 
               'author_5_eng last name':eng_auth_names[4], 
               'author_5_eng initials':eng_auth_initials[4],
               'author_5_eng affiliation':eng_auth_affiliations[4], 

               'author_6_id':final_auths_ordered[5], 
               'author_6_affiliation_id':final_ids_ordered[5],               
               'author_6_ru last name':rus_auth_names[5], 
               'author_6_ru initials':rus_auth_initials[5],
               'author_6_ru affiliation':rus_auth_affiliations[5], 
               'author_6_eng last name':eng_auth_names[5], 
               'author_6_eng initials':eng_auth_initials[5],
               'author_6_eng affiliation':eng_auth_affiliations[5], 

               'author_7_id':final_auths_ordered[6], 
               'author_7_affiliation_id':final_ids_ordered[6],               
               'author_7_ru last name':rus_auth_names[6], 
               'author_7_ru initials':rus_auth_initials[6],
               'author_7_ru affiliation':rus_auth_affiliations[6], 
               'author_7_eng last name':eng_auth_names[6], 
               'author_7_eng initials':eng_auth_initials[6],
               'author_7_eng affiliation':eng_auth_affiliations[6], 

               'author_8_id':final_auths_ordered[7], 
               'author_8_affiliation_id':final_ids_ordered[7],               
               'author_8_ru last name':rus_auth_names[7], 
               'author_8_ru initials':rus_auth_initials[7],
               'author_8_ru affiliation':rus_auth_affiliations[7], 
               'author_8_eng last name':eng_auth_names[7], 
               'author_8_eng initials':eng_auth_initials[7],
               'author_8_eng affiliation':eng_auth_affiliations[7], 
               
               'title_ru':ru_title, 
               'title_eng':eng_title, 
               
               'year':data['root']['item']['year'],
               
               'keywords_ru':ru_keywords,
               'keywords_eng':eng_keywords,
               
               'abstract_ru':ru_abstract,
               'abstract_eng':eng_abstract,
               
               'rubric':data['root']['item']['rubrics']['rubric']['#text'],
               'rubric_code':data['root']['item']['rubrics']['rubric']['@code'],
               'journal_name':data['root']['item']['journal']['#text'],
               'journal_id':data['root']['item']['journal']['@id'],
               'journal_issn':data['root']['item']['journal']['@issn'],
               'journal_eissn':data['root']['item']['journal']['@eissn'],
               'journnal_impact_factor':data['root']['item']['journal']['@impactFactor'],
               'journal_VAK':data['root']['item']['journal']['@VAK'],
               'journal_Scopus':data['root']['item']['journal']['@Scopus'],
               'journal_WebofScience':data['root']['item']['journal']['@WebofScience'],
               'journal_RSCI':data['root']['item']['journal']['@RSCI'],
               'journal_RISC':data['root']['item']['journal']['@RISC'],
               'volume':volume,
               'journal_number':number,
               'journal_pages':jp,
               'number_of_pages':np,
               'publisher_id':publisher_id,
               'publisher':publisher,
               'DOI':code_DOI,
               'EDN':code_EDN,
               'UDK':code_UDK,
               'support': fin,
                               
               'genre_id':data['root']['item']['@genreId'],
               'cited':data['root']['item']['@cited'],
               'corecited':data['root']['item']['@coreCited'],
               'parent_id':data['root']['item']['@parentId'],
               'source_id':data['root']['item']['@sourceId'],
               'isFT':data['root']['item']['@isFT'],
               'isNEW':data['root']['item']['@isNew'],
               'link':data['root']['item']['link'],
               'url':url,
               'ref':data['root']['item']['ref']
              }
    
    
    df = df.append(new_row, ignore_index=True)
    counter += 1
    
    # каждые 110 строк засыпаем на 1.8 секунды чтобы не перегрузить сервис
    if counter % 110 == 0:
        sleep(1.8)
    # каждые 1000 строк сохраняем промежуточную выгрузку
    if counter % 1000 == 0:
        df.to_csv(f'df_7_july_{counter}_rows.csv')
        df.to_excel(f'df_7_july_{counter}_rows.xlsx')

100%|██████████| 18379/18379 [2:32:57<00:00,  2.00it/s]   


Смотрю на данные после (возможного) удаления дупликатов. Ура, уникальных наблюдений действительно 66 437!

In [92]:
df = df.drop_duplicates('id')
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66476,46499800,4,406338,1250,Передня,Дмитрий Григорьевич,Академия управления МВД России,Perednya,D.G.,Management Academy of the Ministry of the Inte...,...,4,0,0,46499795,,1,1,http://elibrary.ru/item.asp?id=46499800,None,"Передня, Д.Г. Интенсификация образовательного ..."
66477,46499861,4,510760,4812,Лескова,Ирина Валерьевна,ФГБОУ ВО «Российский государственный социальны...,Leskova,Irina Valerʹevna,Russian State Social University,...,4,0,0,46499841,,1,1,http://elibrary.ru/item.asp?id=46499861,https://sfk-mn.ru/PDF/24SCSK221.pdf,"Лескова, И.В. Мигранты армяне в российском соц..."
66478,46503369,4,622383,11,Долгов,Александр Юрьевич,none,Dolgov,A.Yu.,none,...,4,0,0,46503368,,1,1,http://elibrary.ru/item.asp?id=46503369,None,"Долгов, А.Ю. Социология смерти: продолжая труд..."
66479,46505702,4,491974,583,Бреславский,Анатолий Сергеевич,"Институт монголоведения, буддологии и тибетоло...",Breslavsky,Anatoly S.,"Institute for Mongolian, Buddhist and Tibetan ...",...,4,0,0,46505698,,1,1,http://elibrary.ru/item.asp?id=46505702,None,"Бреславский, А.С. Территориальное общественное..."


Записываю финальный файл в .csv и .xlsx формате.

In [93]:
df.to_csv('final_data.csv')
df.to_excel('final_data.xlsx')